#GRU Two types : 

*   Single directional
*   Bi-Directional

**N.B : Single directional GRU is described here**



In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install --upgrade gensim

     |████████████████████████████████| 24.2MB 129kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [3]:
import gensim
print(gensim.__version__)

3.8.3


#Model selection catagory - wise
<pre>
1.   <b>Word2vec model :</b>
      -> 'word2vec-google-news-300'
2.   <b>Glove model :</b> 
     *  Twitter Glove Embeddings
        -> 'glove-twitter-25'
        -> 'glove-twitter-50'
        -> 'glove-twitter-100'
        -> 'glove-twitter-200'
     *  Wikipedia Glove Embeddings
        -> 'glove-wiki-gigaword-50'
        -> 'glove-wiki-gigaword-100'
        -> 'glove-wiki-gigaword-200'
        -> 'glove-wiki-gigaword-300'
3.   <b>FastText model :</b>
        -> 'fasttext-wiki-news-subwords-300'
</pre>

In [4]:
import gensim.downloader as api
word2vec_model = api.load('fasttext-wiki-news-subwords-300')

[=================================================-] 100.0% 958.3/958.4MB downloaded


In [5]:
import json
import random
import numpy as np
from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.metrics import TruePositives,FalsePositives,TrueNegatives,FalseNegatives,Precision,Recall,Accuracy,confusion_matrix

In [6]:
def find_best(training_details):
  loss = []
  accuracy = []
  val_loss = []
  val_accuracy = []

  for i in range(len(training_details)):
    loss.append(training_details[i]['loss'])
    accuracy.append(training_details[i]['accuracy'])
    val_loss.append(training_details[i]['val_loss'])
    val_accuracy.append(training_details[i]['val_accuracy'])
  
  max_accuracy = max(accuracy)
  max_accuracy_indices = [i for i, x in enumerate(accuracy) if x == max_accuracy]

  temp_val_accuracy = []
  if len(max_accuracy_indices)>1:
    for i in max_accuracy_indices:
      temp_val_accuracy.append(val_accuracy[i])
    
    max_val_accuracy = max(temp_val_accuracy)

    max_val_accuracy_indices = [i for i, x in enumerate(val_accuracy) if x == max_val_accuracy]
    
    temp_loss = []
    if len(max_val_accuracy_indices)>1:
      for i in max_val_accuracy_indices:
        temp_loss.append(loss[i])
      
      min_loss = min(temp_loss)
      
      min_loss_indices = [i for i, x in enumerate(loss) if x == min_loss]

      temp_val_loss = []

      if len(min_loss_indices)>1:
        for i in min_loss_indices:
          temp_val_loss.append(val_loss[i])
        
        min_val_loss = min(temp_val_loss)

        min_val_loss_indices = [i for i, x in enumerate(val_loss) if x == min_val_loss]

        if len(min_val_loss_indices)>1:
          return random.choice(min_val_loss_indices)
        else:
          return min_val_loss_indices
      else:
        return min_loss_indices
    else:
      return max_val_accuracy_indices
  else:
    return max_accuracy_indices

In [7]:
f = open('gdrive/My Drive/Colab Notebooks/Soft_computing/SoftComputing_project/final_data.json') 
data = json.load(f)
f1 = open('gdrive/My Drive/Colab Notebooks/Soft_computing/SoftComputing_project/class_value.json') 
label = json.load(f1)
# label = np.array(label)

In [8]:
train_and_valid_data, test_data, train_and_valid_label, test_label = train_test_split(data,label, test_size=0.2, random_state=1)
 #now , train and validation data = 0.8 portion data of total 1.0 portion data , now we divide this 0.8 portion data to train and valid data 

In [9]:
train_data,valid_data,train_label,valid_label = train_test_split(train_and_valid_data,train_and_valid_label, test_size=0.25, random_state=1)
# as main data is 0.8 so here train data = 0.8*0.75 = 0.6  and valid data = 0.8* 0.25 = 0.2

In [10]:
print(len(train_data))
print(len(test_data))
print(len(valid_data))

36541
12181
12181


In [11]:
train_label = np.array(train_label)
valid_label = np.array(valid_label)
test_label = np.array(test_label)

In [12]:
vocab_size = 30000
embedding_dim_normal_embedding = 64
max_length = 32
# trunc_type='post'
oov_tok = "<OOV>"

# tokenizer = Tokenizer(num_words = vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_data)
word_index = tokenizer.word_index
training_sequences = tokenizer.texts_to_sequences(train_data)
training_padded = pad_sequences(training_sequences,maxlen=max_length) #truncating=trunc_type

valid_sequences = tokenizer.texts_to_sequences(valid_data)
valid_padded = pad_sequences(valid_sequences,maxlen=max_length) #truncating=trunc_type

testing_sequences = tokenizer.texts_to_sequences(test_data)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)# truncating=trunc_type

In [13]:
total_words = len(word_index)
print(total_words)

23596


In [14]:
embedding_dim_pretrained_embedding = word2vec_model.vector_size
embedding_matrix = np.zeros((vocab_size, embedding_dim_pretrained_embedding))
for word, i in word_index.items():
  try:
    embedding_matrix[i] = word2vec_model[word]
  except:
    pass

In [15]:
# print(total_words)

In [16]:
# len(embedding_matrix[0])

In [17]:
tf.keras.backend.clear_session()

#Tune Batch size

In [ ]:
def param_tune(epoch_num,batch_size_num):
  batch = []
  epoch = []
  training_details = []
  for i in range(len(batch_size_num)):
    # tf.keras.backend.clear_session()
    model3 = tf.keras.Sequential([
                                  tf.keras.layers.Embedding(vocab_size,EMBEDDING_DIM,weights=[embedding_matrix],input_length=max_length,trainable=False),  
                                  tf.keras.layers.GRU(embedding_dim),
                                  # tf.keras.layers.Dropout(0.5),
                                  tf.keras.layers.Dense(1,activation='sigmoid')
                                  ])
    model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    # model3.summary()
    print("Epoch : ",epoch_num,"\tBatch_size : ",batch_size_num[i])
    history = model3.fit(training_padded, train_label, epochs=epoch_num, validation_data=(valid_padded, valid_label),batch_size=batch_size_num[i])
    batch.append(batch_size_num[i]) 
    epoch.append(epoch_num)
    training_details.append({'loss': history.history['loss'][-1],
                             'accuracy':history.history['accuracy'][-1],
                             'val_loss':history.history['val_loss'][-1],
                             'val_accuracy':history.history['val_accuracy'][-1]})
    # del model3
  return batch,epoch,training_details

In [ ]:
e = 10
# b_size = [10,20,30,40,50,60,70,80,90,100]
b_size = [16,32,64,128]
batch,epoch,training_details = param_tune(e,b_size)
for i in range(len(batch)):
  print("\n","Batch : ",batch[i]," Epoch : ",epoch[i]," Details : ",training_details[i])

286/286 [==============================] - 8s 29ms/step - loss: 0.2187 - accuracy: 0.9219 - val_loss: 0.1723 - val_accuracy: 0.9408
Epoch 2/10
286/286 [==============================] - 8s 28ms/step - loss: 0.1586 - accuracy: 0.9475 - val_loss: 0.1624 - val_accuracy: 0.9430
Epoch 3/10
286/286 [==============================] - 8s 28ms/step - loss: 0.1489 - accuracy: 0.9493 - val_loss: 0.1663 - val_accuracy: 0.9438
Epoch 4/10
286/286 [==============================] - 8s 28ms/step - loss: 0.1425 - accuracy: 0.9515 - val_loss: 0.1580 - val_accuracy: 0.9469
Epoch 5/10
286/286 [==============================] - 8s 28ms/step - loss: 0.1352 - accuracy: 0.9537 - val_loss: 0.1576 - val_accuracy: 0.9466
Epoch 6/10
286/286 [==============================] - 8s 29ms/step - loss: 0.1294 - accuracy: 0.9548 - val_loss: 0.1629 - val_accuracy: 0.9453
Epoch 7/10
286/286 [==============================] - 8s 28ms/step - loss: 0.1240 - accuracy: 0.9557 - val_loss: 0.1614 - val_accuracy: 0.9458
Epoch 8/10

In [ ]:
indices = find_best(training_details)
print("Best parameter is : ","Batch : ",batch[indices[0]])

Best parameter is :  Batch :  16


#Tune optimizer

In [ ]:
def param_tune(epoch_num,batch_size_num,optimizer_type):
  optimizers = []
  training_details = []
  for i in range(len(optimizer_type)):
    # tf.keras.backend.clear_session()
    model3 = tf.keras.Sequential([
                                  tf.keras.layers.Embedding(vocab_size,EMBEDDING_DIM,weights=[embedding_matrix],input_length=max_length,trainable=False),  
                                  tf.keras.layers.GRU(embedding_dim),
                                  # tf.keras.layers.Dropout(0.5),
                                  tf.keras.layers.Dense(1,activation='sigmoid')
                                  ])
    model3.compile(loss='binary_crossentropy',optimizer=optimizer_type[i],metrics=['accuracy'])
    # model3.summary()
    print("Optimizer : ",optimizer_type[i])
    history = model3.fit(training_padded, train_label, epochs=epoch_num, validation_data=(valid_padded, valid_label),batch_size=batch_size_num)
    optimizers.append(optimizer_type[i]) 
    training_details.append({'loss': history.history['loss'][-1],
                             'accuracy':history.history['accuracy'][-1],
                             'val_loss':history.history['val_loss'][-1],
                             'val_accuracy':history.history['val_accuracy'][-1]})
    # del model3
  return optimizers,training_details

In [ ]:
e = 10
b_size = 16
opt = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam', 'Ftrl'] 
optimizers,training_details = param_tune(e,b_size,opt)
for i in range(len(optimizers)):
  print("\n","Optimizer :",optimizers[i]," Details : ",training_details[i])

2284/2284 [==============================] - 29s 13ms/step - loss: 0.6756 - accuracy: 0.8090 - val_loss: 0.6671 - val_accuracy: 0.8058
Epoch 2/10
2284/2284 [==============================] - 28s 12ms/step - loss: 0.6569 - accuracy: 0.8091 - val_loss: 0.6372 - val_accuracy: 0.8058
Epoch 3/10
2284/2284 [==============================] - 28s 12ms/step - loss: 0.5699 - accuracy: 0.8091 - val_loss: 0.5084 - val_accuracy: 0.8058
Epoch 4/10
2284/2284 [==============================] - 28s 12ms/step - loss: 0.4797 - accuracy: 0.8091 - val_loss: 0.4681 - val_accuracy: 0.8058
Epoch 5/10
2284/2284 [==============================] - 28s 12ms/step - loss: 0.4461 - accuracy: 0.8091 - val_loss: 0.4348 - val_accuracy: 0.8058
Epoch 6/10
2284/2284 [==============================] - 28s 12ms/step - loss: 0.4075 - accuracy: 0.8114 - val_loss: 0.3920 - val_accuracy: 0.8129
Epoch 7/10
2284/2284 [==============================] - 28s 12ms/step - loss: 0.3608 - accuracy: 0.8329 - val_loss: 0.3433 - val_accura

In [ ]:
indices = find_best(training_details)
print("Best parameter is : ","Optimizer :",optimizers[indices[0]])

Best parameter is :  Optimizer : Nadam


#Tune Dropout

In [ ]:
def param_tune(epoch_num,batch_size_num,optimizer_type,dropout_num):
  dropouts = []
  training_details = []
  for i in range(len(dropout_num)):
    # tf.keras.backend.clear_session()
    model3 = tf.keras.Sequential([
                                  tf.keras.layers.Embedding(vocab_size,EMBEDDING_DIM,weights=[embedding_matrix],input_length=max_length,trainable=False),  
                                  tf.keras.layers.GRU(embedding_dim,dropout=dropout_num[i]),
                                  # tf.keras.layers.Dropout(0.5),
                                  tf.keras.layers.Dense(1,activation='sigmoid')
                                  ])
    model3.compile(loss='binary_crossentropy',optimizer=optimizer_type,metrics=['accuracy'])
    # model3.summary()
    print("Dropout : ",dropout_num[i])
    history = model3.fit(training_padded, train_label, epochs=epoch_num, validation_data=(valid_padded, valid_label),batch_size=batch_size_num)
    dropouts.append(dropout_num[i]) 
    training_details.append({'loss': history.history['loss'][-1],
                             'accuracy':history.history['accuracy'][-1],
                             'val_loss':history.history['val_loss'][-1],
                             'val_accuracy':history.history['val_accuracy'][-1]})
    # del model3
  return dropouts,training_details

In [ ]:
e = 10
b_size = 16
opt = 'Nadam'
drop = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
dropouts,training_details = param_tune(e,b_size,opt,drop)
for i in range(len(dropouts)):
  print("\n","Dropout :",dropouts[i]," Details : ",training_details[i])

2284/2284 [==============================] - 30s 13ms/step - loss: 0.2111 - accuracy: 0.9251 - val_loss: 0.1744 - val_accuracy: 0.9437
Epoch 9/10
2284/2284 [==============================] - 30s 13ms/step - loss: 0.2060 - accuracy: 0.9287 - val_loss: 0.1888 - val_accuracy: 0.9417
Epoch 10/10
2284/2284 [==============================] - 30s 13ms/step - loss: 0.2045 - accuracy: 0.9287 - val_loss: 0.1885 - val_accuracy: 0.9367

 Dropout : 0.1  Details :  {'loss': 0.05958077684044838, 'accuracy': 0.9788456559181213, 'val_loss': 0.22097161412239075, 'val_accuracy': 0.9385929107666016}

 Dropout : 0.2  Details :  {'loss': 0.07132655382156372, 'accuracy': 0.9728524088859558, 'val_loss': 0.20245160162448883, 'val_accuracy': 0.942369282245636}

 Dropout : 0.3  Details :  {'loss': 0.09071385115385056, 'accuracy': 0.9669685959815979, 'val_loss': 0.18430984020233154, 'val_accuracy': 0.9443395733833313}

 Dropout : 0.4  Details :  {'loss': 0.10591717809438705, 'accuracy': 0.9614952802658081, 'val_l

In [ ]:
indices = find_best(training_details)
print("Best parameter is : ","Dropout :",dropouts[indices[0]])

Best parameter is :  Dropout : 0.1


#Tune hidden node

In [ ]:
def param_tune(epoch_num,batch_size_num,optimizer_type,dropout_num,neuron_num):
  neurons_num = []
  training_details = []
  for i in range(len(neuron_num)):
    # tf.keras.backend.clear_session()
    model3 = tf.keras.Sequential([
                                  tf.keras.layers.Embedding(vocab_size,embedding_dim,weights=[embedding_matrix],input_length=max_length,trainable=False), 
                                  tf.compat.v1.keras.layers.CuDNNGRU(neuron_num[i]),
                                  tf.keras.layers.Dropout(dropout_num),
                                  tf.keras.layers.Dense(1,activation='sigmoid')
                                  ])
    model3.compile(loss='binary_crossentropy',optimizer=optimizer_type,metrics=['accuracy'])
    # model3.summary()
    print("Neuron : ",neuron_num[i])
    history = model3.fit(training_padded, train_label, epochs=epoch_num, validation_data=(testing_padded, test_label),batch_size=batch_size_num)
    neurons_num.append(neuron_num[i]) 
    training_details.append({'loss': history.history['loss'][-1],
                             'accuracy':history.history['accuracy'][-1],
                             'val_loss':history.history['val_loss'][-1],
                             'val_accuracy':history.history['val_accuracy'][-1]})
    # del model3
  return neurons_num,training_details

In [ ]:
e = 10
b_size = 16
opt = 'Nadam'
drop = 0.1
neu_ron = [embedding_dim,256,128,64,32,16]
neurons_num,training_details = param_tune(e,b_size,opt,drop,neu_ron)
for i in range(len(neurons_num)):
  print("\n","Neuron :",neurons_num[i]," Details : ",training_details[i])

Neuron :  300
Epoch 1/10
2284/2284 [==============================] - 22s 10ms/step - loss: 0.1747 - accuracy: 0.9425 - val_loss: 0.1536 - val_accuracy: 0.9486
Epoch 2/10
2284/2284 [==============================] - 15s 6ms/step - loss: 0.1460 - accuracy: 0.9508 - val_loss: 0.1452 - val_accuracy: 0.9510
Epoch 3/10
2284/2284 [==============================] - 15s 6ms/step - loss: 0.1352 - accuracy: 0.9533 - val_loss: 0.1495 - val_accuracy: 0.9489
Epoch 4/10
2284/2284 [==============================] - 15s 7ms/step - loss: 0.1224 - accuracy: 0.9565 - val_loss: 0.1523 - val_accuracy: 0.9500
Epoch 5/10
2284/2284 [==============================] - 15s 6ms/step - loss: 0.1044 - accuracy: 0.9618 - val_loss: 0.1645 - val_accuracy: 0.9495
Epoch 6/10
2284/2284 [==============================] - 15s 6ms/step - loss: 0.0829 - accuracy: 0.9700 - val_loss: 0.1748 - val_accuracy: 0.9499
Epoch 7/10
2284/2284 [==============================] - 15s 6ms/step - loss: 0.0631 - accuracy: 0.9771 - val_loss: 

In [ ]:
indices = find_best(training_details)
print("Best parameter is : ","Neuron :",neurons_num[indices[0]])

Best parameter is :  Neuron : 256


#Tune Epoch

In [ ]:
def param_tune(epoch_num,batch_size_num,optimizer_type,dropout_num):
  epochs_num = []
  training_details = []
  global model3
  for i in range(len(epoch_num)):
    # tf.keras.backend.clear_session()
    if i == 0:
      model3 = tf.keras.Sequential([
                                    tf.keras.layers.Embedding(vocab_size,EMBEDDING_DIM,weights=[embedding_matrix],input_length=max_length,trainable=False),  
                                    tf.compat.v1.keras.layers.CuDNNGRU(embedding_dim),
                                    tf.keras.layers.Dropout(dropout_num),
                                    tf.keras.layers.Dense(1,activation='sigmoid')
                                    ])
      model3.compile(loss='binary_crossentropy',optimizer=optimizer_type,metrics=['accuracy'])
      # model3.summary()
      print("Epoch : ",epoch_num[i])
      history = model3.fit(training_padded, train_label, epochs=epoch_num[i], validation_data=(valid_padded, valid_label),batch_size=batch_size_num)
      epochs_num.append(epoch_num[i]) 
      training_details.append({'loss': history.history['loss'][-1],
                              'accuracy':history.history['accuracy'][-1],
                              'val_loss':history.history['val_loss'][-1],
                              'val_accuracy':history.history['val_accuracy'][-1]})
    else:
       print("Epoch : ",epoch_num[i])
       history = model3.fit(training_padded, train_label, epochs=epoch_num[i]-epoch_num[i-1], validation_data=(valid_padded, valid_label),batch_size=batch_size_num)
       epochs_num.append(epoch_num[i]) 
       training_details.append({'loss': history.history['loss'][-1],
                                'accuracy':history.history['accuracy'][-1],
                                'val_loss':history.history['val_loss'][-1],
                                'val_accuracy':history.history['val_accuracy'][-1]})

    # del model3
  return epochs_num,training_details

In [ ]:
e = [5,10,15,20,25,30,35,40,50,60,65,70,75,80,85,95,100]
b_size = 16
opt = 'Nadam'
drop = 0.1
epochs_num,training_details = param_tune(e,b_size,opt,drop)
for i in range(len(epochs_num)):
  print("\n","Epoch :",epochs_num[i]," Details : ",training_details[i])

Epoch :  5
Epoch 1/5
2284/2284 [==============================] - 30s 13ms/step - loss: 0.1769 - accuracy: 0.9410 - val_loss: 0.1753 - val_accuracy: 0.9421
Epoch 2/5
2284/2284 [==============================] - 29s 13ms/step - loss: 0.1482 - accuracy: 0.9499 - val_loss: 0.1564 - val_accuracy: 0.9475
Epoch 3/5
2284/2284 [==============================] - 29s 13ms/step - loss: 0.1375 - accuracy: 0.9522 - val_loss: 0.1575 - val_accuracy: 0.9472
Epoch 4/5
2284/2284 [==============================] - 29s 13ms/step - loss: 0.1281 - accuracy: 0.9551 - val_loss: 0.1550 - val_accuracy: 0.9466
Epoch 5/5
2284/2284 [==============================] - 29s 13ms/step - loss: 0.1175 - accuracy: 0.9579 - val_loss: 0.1656 - val_accuracy: 0.9443
Epoch :  10
Epoch 1/5
2284/2284 [==============================] - 29s 13ms/step - loss: 0.1052 - accuracy: 0.9625 - val_loss: 0.1739 - val_accuracy: 0.9425
Epoch 2/5
2284/2284 [==============================] - 29s 13ms/step - loss: 0.0897 - accuracy: 0.9680 - va

In [ ]:
indices = find_best(training_details)
print("Best parameter is : ","Epoch :",epochs_num[indices[0]])

#Tuning part is end

#Test part now

#At first train model using the final tuned parameter and then test with test data

In [18]:
from tensorflow.keras import backend as K

# def confusion_m_ele(y_true, y_pred):
#     true_positive = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     false_negative = possible_positives - true_positive
#     false_positive = predicted_positives - true_positive
#     true_negative = len(test_data) - true_positive - false_positive - false_negative
#     return true_positive,false_positive,false_negative,true_negative

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
    
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# def confusion_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     false_negatives = possible_positives - true_positives
#     false_positives = predicted_positives - true_positives
#     true_negatives = len(y_true) - true_positives - false_positives - false_negatives
#     return np.array([[true_negatives,false_positives],[false_negatives,true_positives]])

#Bi-direction

#Bi-direction with normal Trainable embedding

In [ ]:
model3 = tf.keras.Sequential([
                                    tf.keras.layers.Embedding(vocab_size,embedding_dim_normal_embedding,input_length=max_length),
                                    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, dropout=0.1, recurrent_dropout=0.1)),
                                    tf.keras.layers.Dense(64,activation = 'relu'),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(1,activation='sigmoid')
                                    ])
model3.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['acc',f1_m,precision_m, recall_m])
model3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 64)            1920000   
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               49920     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 1,978,241
Trainable params: 1,978,241
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model3.fit(training_padded, train_label, epochs=4, validation_data=(valid_padded, valid_label),batch_size=32)

Epoch 1/4
  21/1142 [..............................] - ETA: 3:18 - loss: 0.5825 - acc: 0.8080 - f1_m: 0.0464 - precision_m: 0.0648 - recall_m: 0.0488

KeyboardInterrupt: ignored

In [ ]:
loss, accuracy, f1_score, precision, recall = model3.evaluate(testing_padded,test_label)
print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
# print(loss, accuracy, f1_score, precision, recall)
print('Loss: %.2f' % (loss*100))
print('Accuracy: %.2f' % (accuracy*100))
print('Precision: %.2f' % (precision*100))
print('Recall: %.2f' % (recall*100))
print('F1_score: %.2f' % (f1_score*100))

#Bi-direction with pretrained embedding

In [ ]:
model3 = tf.keras.Sequential([
                                    tf.keras.layers.Embedding(vocab_size,embedding_dim_pretrained_embedding,weights=[embedding_matrix],input_length=max_length,trainable=False),
                                    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, dropout=0.1, recurrent_dropout=0.1)),
                                    tf.keras.layers.Dense(64,activation = 'relu'),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(1,activation='sigmoid')
                                    ])
model3.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['acc',f1_m,precision_m, recall_m])
model3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 300)           9000000   
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               140544    
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 9,148,865
Trainable params: 148,865
Non-trainable params: 9,000,000
_________________________________________________________________


In [ ]:
model3.fit(training_padded, train_label, epochs=4, validation_data=(valid_padded, valid_label),batch_size=32)

Epoch 1/4
1142/1142 [==============================] - 280s 245ms/step - loss: 0.1974 - acc: 0.9313 - f1_m: 0.7793 - precision_m: 0.7939 - recall_m: 0.7972 - val_loss: 0.1703 - val_acc: 0.9441 - val_f1_m: 0.8488 - val_precision_m: 0.8240 - val_recall_m: 0.8972
Epoch 2/4
1142/1142 [==============================] - 278s 244ms/step - loss: 0.1555 - acc: 0.9490 - f1_m: 0.8582 - precision_m: 0.8510 - recall_m: 0.8877 - val_loss: 0.1594 - val_acc: 0.9470 - val_f1_m: 0.8494 - val_precision_m: 0.8504 - val_recall_m: 0.8718
Epoch 3/4
1142/1142 [==============================] - 276s 242ms/step - loss: 0.1511 - acc: 0.9502 - f1_m: 0.8611 - precision_m: 0.8564 - recall_m: 0.8888 - val_loss: 0.1551 - val_acc: 0.9473 - val_f1_m: 0.8551 - val_precision_m: 0.8410 - val_recall_m: 0.8905
Epoch 4/4
1142/1142 [==============================] - 274s 240ms/step - loss: 0.1465 - acc: 0.9508 - f1_m: 0.8682 - precision_m: 0.8598 - recall_m: 0.8992 - val_loss: 0.1542 - val_acc: 0.9460 - val_f1_m: 0.8466 - val

In [ ]:
loss, accuracy, f1_score, precision, recall = model3.evaluate(testing_padded,test_label)
print('Accuracy: %.2f' % (accuracy*100))

381/381 [==============================] - 8s 20ms/step - loss: 0.1475 - acc: 0.9512 - f1_m: 0.8660 - precision_m: 0.8704 - recall_m: 0.8839
Accuracy: 95.12


In [ ]:
# print(loss, accuracy, f1_score, precision, recall)
print('Loss: %.2f' % (loss*100))
print('Accuracy: %.2f' % (accuracy*100))
print('Precision: %.2f' % (precision*100))
print('Recall: %.2f' % (recall*100))
print('F1_score: %.2f' % (f1_score*100))

Loss: 14.75
Accuracy: 95.12
Precision: 87.04
Recall: 88.39
F1_score: 86.60


#Single Direction

#Single direction with normal Trainable embedding

In [ ]:
model3 = tf.keras.Sequential([
                                    tf.keras.layers.Embedding(vocab_size,embedding_dim_normal_embedding,input_length=max_length),
                                    tf.keras.layers.GRU(64, dropout=0.1, recurrent_dropout=0.1),
                                    tf.keras.layers.Dense(64,activation = 'relu'),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(1,activation='sigmoid')
                                    ])
model3.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['acc',f1_m,precision_m, recall_m])
model3.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 32, 64)            1920000   
_________________________________________________________________
gru_4 (GRU)                  (None, 64)                24960     
_________________________________________________________________
dense_8 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 65        
Total params: 1,949,185
Trainable params: 1,949,185
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model3.fit(training_padded, train_label, epochs=4, validation_data=(valid_padded, valid_label),batch_size=32)

Epoch 1/4
1142/1142 [==============================] - 113s 99ms/step - loss: 0.1875 - acc: 0.9382 - f1_m: 0.7980 - precision_m: 0.8029 - recall_m: 0.8225 - val_loss: 0.1572 - val_acc: 0.9481 - val_f1_m: 0.8556 - val_precision_m: 0.8478 - val_recall_m: 0.8890
Epoch 2/4
1142/1142 [==============================] - 112s 98ms/step - loss: 0.1265 - acc: 0.9560 - f1_m: 0.8763 - precision_m: 0.8524 - recall_m: 0.9225 - val_loss: 0.1651 - val_acc: 0.9456 - val_f1_m: 0.8509 - val_precision_m: 0.8407 - val_recall_m: 0.8857
Epoch 3/4
1142/1142 [==============================] - 112s 98ms/step - loss: 0.0963 - acc: 0.9650 - f1_m: 0.9036 - precision_m: 0.8787 - recall_m: 0.9462 - val_loss: 0.1846 - val_acc: 0.9417 - val_f1_m: 0.8370 - val_precision_m: 0.8386 - val_recall_m: 0.8610
Epoch 4/4
1142/1142 [==============================] - 113s 99ms/step - loss: 0.0671 - acc: 0.9747 - f1_m: 0.9292 - precision_m: 0.9218 - recall_m: 0.9501 - val_loss: 0.2291 - val_acc: 0.9362 - val_f1_m: 0.8233 - val_pre

In [ ]:
loss, accuracy, f1_score, precision, recall = model3.evaluate(testing_padded,test_label)
print('Accuracy: %.2f' % (accuracy*100))

381/381 [==============================] - 3s 9ms/step - loss: 0.2210 - acc: 0.9379 - f1_m: 0.8283 - precision_m: 0.8302 - recall_m: 0.8519
Accuracy: 93.79


In [ ]:
# print(loss, accuracy, f1_score, precision, recall)
print('Loss: %.2f' % (loss*100))
print('Accuracy: %.2f' % (accuracy*100))
print('Precision: %.2f' % (precision*100))
print('Recall: %.2f' % (recall*100))
print('F1_score: %.2f' % (f1_score*100))

Loss: 22.10
Accuracy: 93.79
Precision: 83.02
Recall: 85.19
F1_score: 82.83


#Single direction with pretrained embedding

In [19]:
model3 = tf.keras.Sequential([
                                    tf.keras.layers.Embedding(vocab_size,embedding_dim_pretrained_embedding,weights=[embedding_matrix],input_length=max_length,trainable=False),
                                    tf.keras.layers.GRU(64, dropout=0.1, recurrent_dropout=0.1),
                                    tf.keras.layers.Dense(64,activation = 'relu'),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(1,activation='sigmoid')
                                    ])
model3.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['acc',f1_m,precision_m, recall_m])
model3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 300)           9000000   
_________________________________________________________________
gru (GRU)                    (None, 64)                70272     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 9,074,497
Trainable params: 74,497
Non-trainable params: 9,000,000
_________________________________________________________________


In [20]:
model3.fit(training_padded, train_label, epochs=4, validation_data=(valid_padded, valid_label),batch_size=32)

Epoch 1/4
1142/1142 [==============================] - 93s 81ms/step - loss: 0.1986 - acc: 0.9310 - f1_m: 0.7805 - precision_m: 0.7963 - recall_m: 0.7947 - val_loss: 0.1631 - val_acc: 0.9452 - val_f1_m: 0.8405 - val_precision_m: 0.8530 - val_recall_m: 0.8519
Epoch 2/4
1142/1142 [==============================] - 92s 80ms/step - loss: 0.1554 - acc: 0.9482 - f1_m: 0.8551 - precision_m: 0.8505 - recall_m: 0.8821 - val_loss: 0.1622 - val_acc: 0.9470 - val_f1_m: 0.8531 - val_precision_m: 0.8393 - val_recall_m: 0.8892
Epoch 3/4
1142/1142 [==============================] - 92s 80ms/step - loss: 0.1497 - acc: 0.9502 - f1_m: 0.8626 - precision_m: 0.8532 - recall_m: 0.8961 - val_loss: 0.1548 - val_acc: 0.9478 - val_f1_m: 0.8555 - val_precision_m: 0.8432 - val_recall_m: 0.8910
Epoch 4/4
1142/1142 [==============================] - 91s 80ms/step - loss: 0.1465 - acc: 0.9506 - f1_m: 0.8625 - precision_m: 0.8545 - recall_m: 0.8924 - val_loss: 0.1551 - val_acc: 0.9479 - val_f1_m: 0.8530 - val_precisi

In [21]:
loss, accuracy, f1_score, precision, recall = model3.evaluate(testing_padded,test_label)
print('Accuracy: %.2f' % (accuracy*100))

381/381 [==============================] - 3s 9ms/step - loss: 0.1463 - acc: 0.9510 - f1_m: 0.8659 - precision_m: 0.8623 - recall_m: 0.8914
Accuracy: 95.10


In [22]:
# print(loss, accuracy, f1_score, precision, recall)
print('Loss: %.2f' % (loss*100))
print('Accuracy: %.2f' % (accuracy*100))
print('Precision: %.2f' % (precision*100))
print('Recall: %.2f' % (recall*100))
print('F1_score: %.2f' % (f1_score*100))

Loss: 14.63
Accuracy: 95.10
Precision: 86.23
Recall: 89.14
F1_score: 86.59
